# PORTFOLIO OPTIMIZATION ON SOCIAL TRADING


## Il modello fa una ricerca dei pesi migliori che massimizzano una funzione. Tale funzione esprime la differenza dal rendimento atteso del portafoglio e la volatilità di quest'ultimo. Simula i vari pesi, che hanno determinati vincoli, e poi li testa sulla funzione. 

In [3]:
import pandas as pd       
import numpy as np 
import random
df = pd.read_csv('etoro.csv', sep=';')
df.index = df['data']
df = df.dropna(axis=1, how='all')
df = df.drop('data',axis=1).dropna()
df = df[-36:]
col = list(df.columns)
for col_i in col:
    df[col_i] = df[col_i].astype('float')
df

,jaynemesis,openwings,slowestendy,Robert8E,smartandsocial,Ilakha,clahuz,vidinho,cristianan,richiardstroud,...,mimihaha.1,socialinvestor,stockssqueezer,jdominguez,clicktrader,kubyk1975.1,Benschi87,avishaib,swisscashcow,kingpdef
data,,,,,,,,,,,,,,,,,,,,,
201705,33.26,4.41,0.00,3.43,-1.98,0.00,-5.45,0.00,20.24,0.00,...,0.00,0.00,0.00,0.68,0.00,0.00,0.00,0.00,0.00,0.00
201706,35.04,-2.75,0.00,0.00,0.81,0.00,-3.95,9.24,2.19,0.00,...,-5.37,0.00,0.00,0.64,0.00,0.00,0.00,0.00,0.00,0.00
201707,-1.29,3.36,0.00,6.83,1.45,0.00,-1.21,-16.51,-0.13,0.00,...,1.27,0.00,0.00,0.90,0.00,0.00,0.00,0.00,0.00,0.00
201708,25.51,1.79,0.00,0.00,-3.16,0.00,0.69,-1.22,9.77,0.00,...,0.00,0.00,0.00,1.35,0.00,0.00,0.00,0.00,0.00,0.00
201709,-11.01,-1.81,0.00,-1.91,0.62,0.00,0.00,2.24,-2.54,0.00,...,0.00,0.00,0.00,-0.43,0.00,0.00,0.00,0.00,0.00,0.00
201710,4.82,5.46,0.00,6.38,-0.87,0.00,-1.54,-0.27,1.69,0.00,...,0.00,0.00,0.00,-0.28,0.00,0.00,0.00,0.00,0.00,0.00
201711,27.27,7.11,0.00,1.03,-6.02,0.00,1.23,-0.36,9.15,0.00,...,0.00,0.00,0.00,-2.62,0.00,0.00,0.00,0.00,0.00,0.00
201712,23.87,-11.31,0.00,-4.96,0.33,0.00,0.53,8.79,21.49,0.00,...,0.56,0.00,0.00,6.45,0.00,0.00,0.00,0.00,0.00,0.00
201801,-1.20,-6.44,0.00,-7.80,-0.76,0.00,0.62,-0.94,-0.46,0.00,...,8.51,0.00,0.00,-16.00,-1.85,-5.94,0.00,0.61,0.00,2.63


In [70]:
def f(df,k):   # funzione da ottimizzare
    var=df.var()**0.5 # sigma_i
    dff=df.corr()*var # sigma_i*corr_ij
    dff2=dff.copy()
    for coli in col:
        dff2[coli]=dff[coli]*var*k # sigma_i*corr_ii*sigma_j*w_j
    dff2=dff2*k   # sigma_i*corr_ii*sigma_j*w_j*w_i
    #np.fill_diagonal(dff2.values, 0)
    df3=dff2.where(np.tril(np.ones(dff2.shape)).astype(np.bool))

    df3=df3.fillna(0)
    somma1 =(df.mean()*k).sum()
    somma2=sum(var**2*k)+df3.sum().sum()
    return somma1,somma2,somma1 - somma2

#check sul massimo
def check(pesimin, ris, somma1, somma2, valmin, file, capitale,k):
        if ris>valmin:
            pesimin=k
            valmin=ris
            rendatteso=somma1
            rischioatteso=somma2
            out=pd.DataFrame(pesimin[pesimin>0]*capitale)
            out['sim']=r
            out['ris']=ris
            #out.to_csv(file)
            return pesimin,valmin
        else:
            return pesimin, valmin
        


In [131]:
capitale=6760
x=np.zeros(len(col))
pesimin=x
valmin=0
rendatteso=0
rischioatteso=0
for r in range(0,1):
    tot=100+5
    y=[]
    for i in range(0,len(x)):
        a=random.randrange(0, tot, 5) 
        y.append(a/100)
        tot=tot - a
    for j in range(0,50):
        random.shuffle(y)
        k=pd.DataFrame(y)
        k.index=col
        k.columns=['a']
        k=k['a']
        somma1, somma2, ris = f(df,k)
        pesimin, valmin = check(pesimin,ris, somma1, somma2, valmin, 'output.csv', capitale,k)

# Modello più performante per la scelta dei pesi (Gradient Ascent)

In [172]:
capitale=6760
x=np.zeros(len(col))
pesimin=x
valmin=0
rendatteso=0
rischioatteso=0
from scipy.special import softmax

def dw(df,k):
        var=df.var()**0.5
        dff=df.corr()*var
        dff2=dff.copy()
        for coli in col:
            dff2[coli]=dff[coli]*var*k
        df3=dff2.where(np.tril(np.ones(dff2.shape)).astype(np.bool))

        df3=df3.fillna(0)
        somma1=df.mean() 
        somma2= 2*k*df.var()**0.5**2- df3.sum()
        return somma1, somma2
    


for r in range(0,1):
    tot=100+5
    y=[]
    for i in range(0,len(x)):
        a=random.randrange(0, tot, 5) 
        y.append(a/100)
        tot=tot - a
    for j in range(0,50):
        random.shuffle(y)
        k=pd.DataFrame(y)
        k.index=col
        k.columns=['a']
        k=k['a']

        
somma1, somma2, ris = f(df,k)
print(ris)
for g in range(0,100):   

        somma1, somma2 = dw(df,k)
        k = k + 0.05*(somma1 -somma2)
        somma1, somma2, ris = f(df,k)
        print(ris)
val = softmax(k.values) 
k=pd.DataFrame(val)
k.index=col
k.columns=['a']
k=k['a']
k

-65.58944707857141
-4988.514688321605
-1524782.7763059302
-1457071454.754629
-1893001377827.4548
-2595473446325217.0
-3.5976375785688207e+18
-4.99817455419184e+21
-6.946094321488778e+24
-9.652178440696623e+27
-1.3410578073880165e+31
-1.863037072400491e+34
-2.587993396424805e+37
-3.5948781905360413e+40
-4.993353040635307e+43
-6.93573280006671e+46
-9.633575082928596e+49
-1.338072268923358e+53
-1.8585308577308965e+56
-2.5814209282453712e+59
-3.5854788590499467e+62
-4.9800656529559095e+65
-6.917078447855427e+68
-9.607495031591875e+71
-1.3344353376522085e+75
-1.8534669106892556e+78
-2.5743767324951226e+81
-3.575685738414697e+84
-4.966455728632488e+87
-6.8981682954474345e+90
-9.581224081727253e+93
-1.3307859457088212e+97
-1.8483976678553162e+100
-2.567335445231942e+103
-3.565905427704242e+106
-4.952871090034667e+109
-6.879299665953294e+112
-9.555016275141879e+115
-1.3271457913995752e+119
-1.8433416549271925e+122
-2.560312874798604e+125
-3.556151405215943e+128
-4.939323213512686e+131
-6.86048

jaynemesis        0.0
openwings         1.0
slowestendy       0.0
Robert8E          0.0
smartandsocial    0.0
Ilakha            0.0
clahuz            0.0
vidinho           0.0
cristianan        0.0
richiardstroud    0.0
marek1198         0.0
andrefranklin     0.0
darekwa           0.0
ghalex            0.0
kubyk1975         0.0
simonyung         0.0
infinity0683      0.0
2016dennis        0.0
mimihaha          0.0
vasultoro         0.0
tongschef         0.0
misterg23         0.0
richardstroud     0.0
rayvahey          0.0
johnliontw        0.0
daniel4653        0.0
jakey69           0.0
marek1198.1       0.0
harshmit          0.0
mimihaha.1        0.0
socialinvestor    0.0
stockssqueezer    0.0
jdominguez        0.0
clicktrader       0.0
kubyk1975.1       0.0
Benschi87         0.0
avishaib          0.0
swisscashcow      0.0
kingpdef          0.0
Name: a, dtype: float64

# Modello più performante per la scelta dei pesi (Gradient Ascent con tensorflow)
